In [2]:
import re
import emoji
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from random import choices

### Universal Consts

In [111]:
emoji.emojize('Python is :thumbs_up:', use_aliases = True)

'Python is 👍'

In [112]:
emoji.demojize('🤣', language = 'es')

':cara_revolviéndose_de_la_risa:'

In [56]:
mappings = {'feliz': ['😀','😃','😄','😁']
           ,'risa': ['🤣','😂']
           ,'amor': ['😍','😘','🥰','❤️','🖤']
           ,'burla': ['😛','😜','🤪','😝']
           ,'serio': ['😐','😑','😶','🙄']
           ,'disgusto': ['😒','🙃','😔','🙁','☹️']
           ,'sorpresa': ['😮','😯','😲']        
           ,'miedo': ['😨','😰','😱','😖']
           ,'tristeza': ['😥','😢','😭']
           ,'enojo': ['😡','😠']
           ,'desafio': ['😎','😏']
           ,'vacuna': ['💉']
           ,'enfermedad': ['💀','☠️','🦠']
           ,'oov': ['']
        }

In [92]:
emoji_dictionary = {'feliz': '😃'
                   ,'risa': '😂'
                   ,'amor': '🥰'
                   ,'burla': '😜'
                   ,'serio': '😐'
                   ,'disgusto': '😒'
                   ,'sorpresa': '😮'
                   ,'miedo': '😰'
                   ,'tristeza': '😭'
                   ,'enojo': '😠'
                   ,'desafio': '😎'
                   ,'vacuna': '💉'
                   ,'enfermedad': '🦠'
                   ,'oov': ''
                }

### Getting distinct emojis

In [133]:
path = './input/emojiData/sampleData.csv'

In [134]:
data = pd.read_table(path, encoding = 'utf-8', sep = ',')
data.head()

,created_at,text,retweet_count,source,user_created_at,user_followers_count,user_friends_count,user_location,user_name
0,3/13/2020,@prensa libre vez pasado cuarto hora wall stre...,1,Pota APP,4/29/2017 6:45,1446,0,Guatemala,Noticias de Guatemala
1,3/13/2020,@publinewsgt #nacionales medida prevencion cov...,2,Pota APP,4/29/2017 6:45,1446,0,Guatemala,Noticias de Guatemala
2,3/13/2020,@emisorasunidas video coronavirus turistas est...,1,Pota APP,4/29/2017 6:45,1446,0,Guatemala,Noticias de Guatemala
3,3/13/2020,@prensa libre @tododeportes pl premier league ...,2,Pota APP,4/29/2017 6:45,1446,0,Guatemala,Noticias de Guatemala
4,3/13/2020,@prensa libre @tododeportes pl coronavirus par...,2,Pota APP,4/29/2017 6:45,1446,0,Guatemala,Noticias de Guatemala


In [135]:
data.dropna(subset=['text'], axis = 0, inplace = True)
data = data[['text']]
data.head()

,text
0,@prensa libre vez pasado cuarto hora wall stre...
1,@publinewsgt #nacionales medida prevencion cov...
2,@emisorasunidas video coronavirus turistas est...
3,@prensa libre @tododeportes pl premier league ...
4,@prensa libre @tododeportes pl coronavirus par...


### Just for demo

In [136]:
mappings_vals = list(mappings.values())
emoji_list = [emoji_icon for sublist in mappings_vals for emoji_icon in sublist]
emoji_list[0:3]

['😀', '😃', '😄']

In [137]:
example_emojis = choices(emoji_list, k = len(data))
example_emojis[0:3]

['☠️', '😄', '😂']

In [138]:
data['emojis'] = example_emojis
data.head()

,text,emojis
0,@prensa libre vez pasado cuarto hora wall stre...,☠️
1,@publinewsgt #nacionales medida prevencion cov...,😄
2,@emisorasunidas video coronavirus turistas est...,😂
3,@prensa libre @tododeportes pl premier league ...,😑
4,@prensa libre @tododeportes pl coronavirus par...,😒


In [139]:
data['comments'] = data[['text', 'emojis']].agg(' '.join, axis = 1)
data.head()

,text,emojis,comments
0,@prensa libre vez pasado cuarto hora wall stre...,☠️,@prensa libre vez pasado cuarto hora wall stre...
1,@publinewsgt #nacionales medida prevencion cov...,😄,@publinewsgt #nacionales medida prevencion cov...
2,@emisorasunidas video coronavirus turistas est...,😂,@emisorasunidas video coronavirus turistas est...
3,@prensa libre @tododeportes pl premier league ...,😑,@prensa libre @tododeportes pl premier league ...
4,@prensa libre @tododeportes pl coronavirus par...,😒,@prensa libre @tododeportes pl coronavirus par...


In [140]:
data.iloc[0,-1]

'@prensa libre vez pasado cuarto hora wall street mantuvo perdidas niveles similares ☠️'

### Get Emoji data

In [141]:
def map_emojis(emoji_input):
    emotion = ''
    emoji_icon = ''
    for key, sublist in mappings.items():
        if emoji_input in sublist:
            emotion = key
            emoji_icon = emoji_dictionary[key]
    return emotion, emoji_icon

In [142]:
def extract_emojis(text):
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI['es']]
    first_emoji = emoji_list[0] if len(emoji_list) > 0 else ''
    emotion, emoji_icon = map_emojis(first_emoji)
    return len(emoji_list), emotion, emoji_icon

In [143]:
sample_text = '@Prensa liBre vez 10 de 🤣 pasado es en la cuarto #hora wall street mantuvo perdidas niveles O/ similares. 🤪'
#sample_text = '@Prensa liBre vez 10 de pasado es en la cuarto #hora wall street mantuvo perdidas niveles O/ similares'

In [144]:
extract_emojis(sample_text)

(2, 'risa', '😂')

In [146]:
data[['emoji_count', 'emotion', 'emoji_icon']] = data.apply(lambda row: extract_emojis(row['comments']), axis = 1, result_type = 'expand')
data = data[['comments', 'emoji_count', 'emotion', 'emoji_icon']]
data.head()

,comments,emoji_count,emotion,emoji_icon
0,@prensa libre vez pasado cuarto hora wall stre...,0,oov,
1,@publinewsgt #nacionales medida prevencion cov...,1,feliz,😃
2,@emisorasunidas video coronavirus turistas est...,1,risa,😂
3,@prensa libre @tododeportes pl premier league ...,1,serio,😐
4,@prensa libre @tododeportes pl coronavirus par...,1,disgusto,😒


### NLP

In [10]:
all_stopwords = stopwords.words('spanish')

In [147]:
def normalizar(doc):
    txt = re.sub(r'[^A-z0-9\s{1}áéíóúüñÁÉÍÓÚÜ]', '', doc).lower().strip().rstrip('\n').rstrip('\r\n')
    return txt

In [170]:
def removeStopWords(text, stopwords):
    output = ' '.join([token for token in text.split(' ') if not token in stopwords])
    return output

In [171]:
#data['norm_content'] = list(map(normalizar, data['comments']))
#data['token_content'] = list(map(removeStopWords, data['norm_content'], all_stopwords))

In [172]:
data['comments'] = list(map(normalizar, data['comments']))
data['comments'] = list(map(removeStopWords, data['comments'], all_stopwords))

,comments,emoji_count,emotion,emoji_icon,norm_content,token_content
0,@prensa libre vez pasado cuarto hora wall stre...,0,oov,,prensa libre vez pasado cuarto hora wall stree...,prensa libre vez pasado cuarto hora wall stree...
1,@publinewsgt #nacionales medida prevencion cov...,1,feliz,😃,publinewsgt nacionales medida prevencion covid...,publinewsgt nacionales medida prevencion covid...
2,@emisorasunidas video coronavirus turistas est...,1,risa,😂,emisorasunidas video coronavirus turistas esta...,emisorasunidas video coronavirus turistas esta...
3,@prensa libre @tododeportes pl premier league ...,1,serio,😐,prensa libre tododeportes pl premier league in...,prensa libre tododeportes pl premier league in...
4,@prensa libre @tododeportes pl coronavirus par...,1,disgusto,😒,prensa libre tododeportes pl coronavirus parti...,prensa libre tododeportes pl coronavirus parti...


### Extra

In [173]:
from sklearn.preprocessing import OneHotEncoder

In [177]:
emotions = list(emoji_dictionary.keys())
emotions = np.array(emotions)
emotions = np.expand_dims(emotions, axis = 1)
emotions.shape

(14, 1)

In [180]:
emotions

array([['feliz'],
       ['risa'],
       ['amor'],
       ['burla'],
       ['serio'],
       ['disgusto'],
       ['sorpresa'],
       ['miedo'],
       ['tristeza'],
       ['enojo'],
       ['desafio'],
       ['vacuna'],
       ['enfermedad'],
       ['oov']], dtype='<U10')

In [178]:
ohe = OneHotEncoder()
labels = ohe.fit_transform(emotions).toarray()
labels

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [179]:
ohe.categories_

[array(['amor', 'burla', 'desafio', 'disgusto', 'enfermedad', 'enojo',
        'feliz', 'miedo', 'oov', 'risa', 'serio', 'sorpresa', 'tristeza',
        'vacuna'], dtype='<U10')]

In [185]:
ohe.transform([['feliz']]).toarray()

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [183]:
ohe.inverse_transform(labels)

array([['feliz'],
       ['risa'],
       ['amor'],
       ['burla'],
       ['serio'],
       ['disgusto'],
       ['sorpresa'],
       ['miedo'],
       ['tristeza'],
       ['enojo'],
       ['desafio'],
       ['vacuna'],
       ['enfermedad'],
       ['oov']], dtype='<U10')